# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
# model_name_or_path = 'THUDM/chatglm3-6b'
model_name_or_path = '/home/sunyaofei3/huggingface/models/chatglm3-6b/'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [6]:
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
#                                           trust_remote_code=True,
#                                           revision='b098244')
tokenizer = AutoTokenizer.from_pretrained('/home/sunyaofei3/huggingface/tokenizer/chatglm3-6b', trust_remote_code=True)

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


## 使用原始 ChatGLM3-6B 模型

In [7]:
input_text = "解释下乾卦是什么？"

In [8]:
response, history = base_model.chat(tokenizer, query=input_text)

In [9]:
print(response)

乾卦是《易经》中的一个卦象，它是由两个阴爻夹一个阳爻构成，象征着天、干燥、强健、积极等象征。乾卦的卦辞是“元、亨、利、贞”，意味着“至善、至通、至利、至正”。

乾卦的含义非常丰富，它不仅代表着天、干燥、强健等象征，也象征着变化、进步、领导等象征。在《易经》中，乾卦被认为是一个阳刚之卦，代表着一种积极向上的力量和态度。

乾卦还有一个重要的象征意义，就是“天行健，君子以自强不息”。这意味着，君子应该效仿天的行健，不断自强，不断前进，以达到至善、至通、至利、至正的目标。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [10]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的一个卦象，它是八卦之一，由两个阳爻夹一个阴爻构成。讼卦象征着诉讼、争端、矛盾等象征。

讼卦的卦辞是“元、亨、利、贞”，意味着“至善、至通、至利、至正”。它表示，在争端和诉讼中，君子应该以正义为依据，以和平为宗旨，以诚实为原则，以公益为出发点，才能最终达到至善、至通、至利、至正的目标。

从周易的角度来看，讼卦的真正含义并不是 simply about resolving disputes or winning court cases. Instead, it represents the ongoing process of maintaining justice and righteousness in all aspects of life. This includes resolving conflicts, promoting fairness, and striving for a more just and harmonious society.

因此，讼卦是一个 call to action, encouraging all individuals and organizations to strive for what is right and just, even in the face of challenges and disputes.


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [11]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240225_222843"

# peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"
peft_model_path = "/home/sunyaofei3/huggingface/models/chatglm3-6b-epoch3"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [12]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [13]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。其卦辞为“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。”

在传统解卦中，乾卦预示着大吉大利，事业如日中天，但也提醒要警惕盛极必衰的道理。经商方面顺利发展，但要冷静分析形势，坚持商业道德。对于婚恋，尽管阳盛阴衰，但刚柔可相济，最终形成美满结果。总体而言，乾卦代表着充满活力和力量的时机。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240225_222843）：
[gMASK] sop 解释下乾卦是什么？ 在周易中，乾卦代表着天（刚健）的特性，具有刚健强劲的品质。它由六个阳爻组成，象征着天地的运行，以及君子观众的积极入世。它预示着先难后易，先困后亨，最终利见天命。同时，它也提醒我们，天地的运行是刚健的，君子应该积极入世，并且坚持刚健强劲的品质。


In [14]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦"上卦为乾(天)，下卦为坎(水)"，是卦象中的一种卦。在卜卦中，讼卦预示着事态最终会向好的方向发展，但需要警惕风险。意味着书面文字可以感化人心，但前行时需谨小慎微。

讼卦的核心哲学是：天行健，君子以自强不息。君子观此卦象，取法天行，自強力，以志自勉。

讼卦的卦辞这样描述：

象言：天行健，君子以自强不息。

卦象：上卦为乾(天)，下卦为坎(水)

variation：

- 初爻为阳爻（天），表示开始时的情况
- 二爻为阴爻（地），表示情况尚好
- 三爻为阳爻（天），表示情况仍好，仍需自强
- 初爻为阳爻（天），表示开始时的情况

总结：讼卦预示着事态最终会向好的方向发展，但需要警惕风险。意味着书面文字可以感化人心，但前行时需谨小慎微。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240225_222843）：
[gMASK] sop 周易中的讼卦是什么卦象

 在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


In [15]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，得师卦者将面临困难重重，忧心劳众，宜包容别人，艰苦努力，摒除一切困难。台湾国学大儒傅佩荣解则提到，对于时运、财运、家宅和身体等方面会有相应影响。


传统解卦认为，师卦具有养兵聚众、出师攻伐之象，彼此有伤，难得安宁的大象。在运势方面，预示着困难重重，需要以正规行事，谨小慎微，严于律已。在事业、经商、求名、婚恋和决策等方面，都需要保持冷静、谨慎，注意避免敌人和困难带来的不利影响，必能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240225_222843）：
[gMASK] sop 师卦是什么？ 师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，得师卦者将面临困难重重，忧心劳众，宜包容别人，艰苦努力，摒除一切困难。台湾国学大儒傅佩荣解则提到，对于时运、财运、家宅和身体等方面会有相应影响。


传统解卦认为，师卦具有养兵聚众、出师攻伐之象，彼此有伤，难得安宁的大象。在运势方面，预示着困难重重，需要以正规行事，谨小慎微，严于律已。在事业、经商、求名、婚恋和决策等方面，都需要保持冷静、谨慎，注意避免敌人和困难带来的不利影响，必能成功。
